In [1]:
# Importing libraries
import os
import pandas as pd
import json
from sqlalchemy import create_engine

In [ ]:
# Reading the data
data=open('sample_airbnb.json','r')
airbnb=json.load(data)
airbnb

In [4]:
# Function to extract and transform Hotels information
def extract_hotels_info(airbnb):
    hotels = []
    for i in airbnb:
        hotel_data = {
            'id': i['_id'],
            'listing_url': i['listing_url'],
            'name': i['name'],
            'description': i.get('description'),
            'neighborhood': i.get('neighborhood_overview'),
            'transit': i.get('transit'),
            'street': i['address'].get('street'),
            'suburb': i['address'].get('suburb'),
            'government_area': i['address'].get('government_area'),
            'market': i['address'].get('market'),
            'country': i['address'].get('country'),
            'country_code': i['address'].get('country_code'),
            # Formatting the coordinates as 'latitude, longitude'
            'coordinates': ', '.join(map(str, i['address'].get('location', {}).get('coordinates', [0.0, 0.0])))
        }
        hotels.append(hotel_data)
    return pd.DataFrame(hotels)

hotels_df = extract_hotels_info(airbnb)

# Checking for Missing Values
missing_values = hotels_df.isnull().sum()
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = hotels_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Missing Values per Column:
 id                 0
listing_url        0
name               0
description        0
neighborhood       0
transit            0
street             0
suburb             0
government_area    0
market             0
country            0
country_code       0
coordinates        0
dtype: int64
Number of duplicate rows: 0


In [3]:
# Function to extract pricing information
def extract_pricing_info(airbnb):
    price =[]
    for i in airbnb:
        price_data ={ 
            'price': int(i.get('price', 0) or 0),  
            'weekly_price': int(i.get('weekly_price', 0) or 0), 
            'monthly_price': int(i.get('monthly_price', 0) or 0), 
            'security_deposit': int(i.get('security_deposit', 0) or 0),  
            'cleaning_fee': int(i.get('cleaning_fee', 0) or 0),  
            'cancellation_policy': i.get('cancellation_policy', 'N/A')  
        }
        price.append(price_data)
    return pd.DataFrame(price) 

price_df = extract_pricing_info(airbnb)

# Checking for Missing Values
missing_values = price_df.isnull().sum() 
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = price_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Dropping duplicates
if duplicates > 0:
    price_df = price_df.drop_duplicates()
    print("Duplicates dropped.")
else:
    print("No duplicates found.")


Missing Values per Column:
 price                  0
weekly_price           0
monthly_price          0
security_deposit       0
cleaning_fee           0
cancellation_policy    0
dtype: int64
Number of duplicate rows: 1136
Duplicates dropped.


In [5]:
# Function to extract and transform Property information
def extract_property_info(airbnb):
    rooms = []
    for i in airbnb:
        room_data = {
            'id': i['_id'],
            'images': i['images']['picture_url'] if 'images' in i and 'picture_url' in i['images'] else '',
            'property_type': i.get('property_type', 'N/A'),
            'room_type': i.get('room_type', 'N/A'),
            'bed_type': i.get('bed_type', 'N/A'),
            'min_nights': i.get('minimum_nights', 'N/A'),
            'max_nights': i.get('maximum_nights', 'N/A'),
            'accommodates': int(i.get('accommodates', 0) or 0), 
            'bedrooms': int(i.get('bedrooms', 0) or 0),  
            'beds': int(i.get('beds', 0) or 0),  
            'bathrooms': float(i.get('bathrooms', 0.0) or 0.0),  
            'amenities': ', '.join(i.get('amenities', [])),  
            'guests_included': int(i.get('guests_included', 0) or 0)  
        }
        rooms.append(room_data)
    return pd.DataFrame(rooms)

property_df = extract_property_info(airbnb)

# Checking for Missing Values
missing_values = property_df.isnull().sum() 
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = property_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Missing Values per Column:
 id                 0
images             0
property_type      0
room_type          0
bed_type           0
min_nights         0
max_nights         0
accommodates       0
bedrooms           0
beds               0
bathrooms          0
amenities          0
guests_included    0
dtype: int64
Number of duplicate rows: 0


In [6]:
# Function to extract the availability details
def extract_availability_details(airbnb):
    availability = []
    for i in airbnb:
        avail_data = {
            'availability_30': int(i['availability'].get('availability_30', 0)),
            'availability_60': int(i['availability'].get('availability_60', 0)),
            'availability_90': int(i['availability'].get('availability_90', 0)),
            'availability_365': int(i['availability'].get('availability_365', 0))
        }
        availability.append(avail_data)
    return pd.DataFrame(availability)

availability_df = extract_availability_details(airbnb)

# Checking for Missing Values
missing_values = availability_df.isnull().sum() 
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = availability_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Dropping duplicates
if duplicates > 0:
    availability_df = availability_df.drop_duplicates()
    print("Duplicates dropped.")
else:
    print("No duplicates found.")


Missing Values per Column:
 availability_30     0
availability_60     0
availability_90     0
availability_365    0
dtype: int64
Number of duplicate rows: 2684
Duplicates dropped.


In [7]:
# Function to extract and transform Host information
def extract_host_info(airbnb):
    hosts = []
    for i in airbnb:
        host_data = {
            'id': i['_id'],
            'host_id': i['host']['host_id'],
            'host_url': i['host']['host_url'],
            'host_name': i['host']['host_name'],
            'host_location': i['host'].get('host_location', ''),
            'host_about': i['host'].get('host_about', ''),
            'host_neighborhood': i['host'].get('host_neighbourhood', ''),
            'host_picture_url': i['host'].get('host_picture_url', ''),
            'host_response_rate': int(i['host'].get('host_response_rate', 0)),
            'host_total_listings': int(i['host'].get('host_total_listings_count', 0)),
            'host_verifications': ', '.join(i['host'].get('host_verifications', []))
        }
        hosts.append(host_data)
    return pd.DataFrame(hosts)

hosts_df = extract_host_info(airbnb)

# Checking for Missing Values
missing_values = hosts_df.isnull().sum() 
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = hosts_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Missing Values per Column:
 id                     0
host_id                0
host_url               0
host_name              0
host_location          0
host_about             0
host_neighborhood      0
host_picture_url       0
host_response_rate     0
host_total_listings    0
host_verifications     0
dtype: int64
Number of duplicate rows: 0


In [8]:
# Function to extract Review scores information
def extract_reviews_scores(airbnb):
    review_scores = []
    for i in airbnb:
        review_data = {
            'id': i['_id'],
            'first_review': i.get('first_review', 'N/A'),
            'last_review': i.get('last_review', 'N/A'),
            'num_reviews': int(i.get('number_of_reviews', 0)),
            'overall_score': int(i['review_scores'].get('review_scores_accuracy', 0)),
            'cleanliness_score': int(i['review_scores'].get('review_scores_cleanliness', 0)),
            'checkin_score': int(i['review_scores'].get('review_scores_checkin', 0)),
            'communication_score': int(i['review_scores'].get('review_scores_communication', 0)),
            'location_score': int(i['review_scores'].get('review_scores_location', 0)),
            'value_score': int(i['review_scores'].get('review_scores_value', 0)),
            'rating': float(i['review_scores'].get('review_scores_rating', 0)),
            'weekly_price': float(i.get('weekly_price', 0)),  
            'monthly_price': float(i.get('monthly_price', 0))
        }
        review_scores.append(review_data)
    return pd.DataFrame(review_scores)

review_scores_df= extract_reviews_scores(airbnb)

# Checking for Missing Values
missing_values = review_scores_df.isnull().sum() 
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = review_scores_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")



Missing Values per Column:
 id                     0
first_review           0
last_review            0
num_reviews            0
overall_score          0
cleanliness_score      0
checkin_score          0
communication_score    0
location_score         0
value_score            0
rating                 0
weekly_price           0
monthly_price          0
dtype: int64
Number of duplicate rows: 0


In [9]:
# Function to extract reviews information
def extract_review_info(airbnb):
    reviews = []
    for listing in airbnb:
        r_list = listing.get('reviews', [])
        for review in r_list:  
            review_data = {
                'comment_id': review.get('_id', 'Unknown'),
                'date': review.get('date', '1970-01-01'),
                'reviewer_id': review.get('reviewer_id', 'Unknown'),
                'reviewer_name': review.get('reviewer_name', 'Anonymous'),
                'comments': review.get('comments', 'No comments')
            }
            reviews.append(review_data)
    return pd.DataFrame(reviews)

reviews_df= extract_review_info(airbnb)

# Checking for Missing Values
missing_values = reviews_df.isnull().sum() 
print("Missing Values per Column:\n", missing_values)

# Checking for Duplicates
duplicates = reviews_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Missing Values per Column:
 comment_id       0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64
Number of duplicate rows: 0


In [10]:
import pymysql
# Creating a Database
mydb = pymysql.connect(
    user='root',
    password='12345678',
    host='localhost'
)
mycursor = mydb.cursor()
mycursor.execute("Create database airbnb")

1

In [11]:
# Function to establish database connection
def create_db_connection(user, password, host, database):
    connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    engine = create_engine(connection_string)
    return engine

# Function to store DataFrame in MySQL
def store_dataframe_to_mysql(df, table_name, engine):
    df.to_sql(name=table_name, con=engine, index=False, if_exists='replace')
    print(f"Data stored in {table_name} table successfully.")

# database credentials
user = 'root'
password = '12345678'
host = 'localhost'  
database = 'airbnb'

# Creating database connection
engine = create_db_connection(user, password, host, database)

# Storing each DataFrame in MySQL
store_dataframe_to_mysql(hotels_df, 'hotels', engine)
store_dataframe_to_mysql(price_df, 'pricing', engine)
store_dataframe_to_mysql(property_df, 'properties', engine)
store_dataframe_to_mysql(availability_df, 'availability', engine)
store_dataframe_to_mysql(hosts_df, 'hosts', engine)
store_dataframe_to_mysql(review_scores_df, 'review_scores', engine)
store_dataframe_to_mysql(reviews_df, 'reviews', engine)


Data stored in hotels table successfully.
Data stored in pricing table successfully.
Data stored in properties table successfully.
Data stored in availability table successfully.
Data stored in hosts table successfully.
Data stored in review_scores table successfully.
Data stored in reviews table successfully.
